In [4]:
! pip install youtube-transcript-api

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import json

with open('C:\\Users\\User\\Desktop\\hamza2.json') as f:
    data = json.load(f)

df = pd.DataFrame(data) 
df['transcript'] = None

print(df.head())

                        video_title     video_id    channel_name  \
0  انتهاكات ومجازر في الساحل السوري  Jw0qhd3JHtc  Ahmad Fakhouri   
1  انتهاكات ومجازر في الساحل السوري  Jw0qhd3JHtc  Ahmad Fakhouri   
2  انتهاكات ومجازر في الساحل السوري  Jw0qhd3JHtc  Ahmad Fakhouri   
3  انتهاكات ومجازر في الساحل السوري  Jw0qhd3JHtc  Ahmad Fakhouri   
4  انتهاكات ومجازر في الساحل السوري  Jw0qhd3JHtc  Ahmad Fakhouri   

                                          video_tags  \
0  أحمد الشرع, بشار الأسد, أخبار سوريا, الانتهاكا...   
1  أحمد الشرع, بشار الأسد, أخبار سوريا, الانتهاكا...   
2  أحمد الشرع, بشار الأسد, أخبار سوريا, الانتهاكا...   
3  أحمد الشرع, بشار الأسد, أخبار سوريا, الانتهاكا...   
4  أحمد الشرع, بشار الأسد, أخبار سوريا, الانتهاكا...   

                                             comment  likes_on_comment  \
0  انا كنت أحد متابعي قناة الجزيرة و كنت اعتبرها ...               906   
1  كلامك يحمل بين طياته تبرير للجرائم و المجازر و...               345   
2  لعن الله كل من طمس الحقيقة وا

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi

transcript_cache = {}

def get_transcript_cached(video_id):
    if video_id in transcript_cache:
        return transcript_cache[video_id] 

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ar'])
        text = " ".join([segment['text'] for segment in transcript])
        transcript_cache[video_id] = text  # Cache it
        return text
    except Exception:
        transcript_cache[video_id] = None
        return None


In [3]:
for idx, row in df.iterrows():
    if pd.isnull(row['transcript']):
        video_id = row['video_id']
        df.at[idx, 'transcript'] = get_transcript_cached(video_id)

In [8]:
missing_transcripts = df[df['transcript'].isnull()]

video_ids_missing = missing_transcripts['video_id'].unique()

print(video_ids_missing)

['B_Mx4JRmGro' 'BsKX8OM0Cq4' 'yeMbo4KI8RM' 'TuO2ON3Nla0' 'a0v9cB6R9xY'
 'qpQwbE4TpX8' 'B1aKhni0Lzg' 'LXerUX1Awmo' 'YNPCqiq1Ubk' 'AV05KzyRgiY'
 'gPCAqEwG8us' 'ON7V2XiQFrM' '0Ro2djOERr4']


In [5]:
df['transcript'].isna().sum() 

594

In [11]:
df = df.dropna(subset=['transcript']) 

In [ ]:
! pip install frasa stanza arabic-reshaper python-bidi pandas pyarabic
! python -m stanza.download -d ar  # Download Arabic models for Stanza

In [8]:
! pip install stanza farasapy pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from farasa.ner import FarasaNamedEntityRecognizer

# Initialize Farasa tools (requires API key)
segmenter = FarasaSegmenter(interactive=True)  # for word segmentation
stemmer = FarasaStemmer(interactive=True)     # for stemming
ner = FarasaNamedEntityRecognizer(interactive=True)  # for NER

In [ ]:
import stanza

stanza.download('ar')
nlp = stanza.Pipeline('ar', processors='tokenize,lemma,pos,ner')

In [ ]:
import re
import pandas as pd
from farasa.segmenter import FarasaSegmenter

def clean_arabic_text(text):
    if not isinstance(text, str):
        return ""
    
    # Remove diacritics (harakat)
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)
    
    # Normalize Alef variations (أ, إ, آ → ا)
    text = re.sub(r'[أإآ]', 'ا', text)
    
    # Remove extra whitespace and non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply cleaning to the transcript column
df['cleaned_transcript'] = df['transcript'].apply(clean_arabic_text) 


In [ ]:
df['cleaned_transcript']

346     عام ميلادي في الجزيره العربيه وفي خضم نار الحر...
347     عام ميلادي في الجزيره العربيه وفي خضم نار الحر...
348     عام ميلادي في الجزيره العربيه وفي خضم نار الحر...
349     عام ميلادي في الجزيره العربيه وفي خضم نار الحر...
350     عام ميلادي في الجزيره العربيه وفي خضم نار الحر...
                              ...                        
3513    خامس اكبر اسباب الضعف الجنسي خامس احسن حلول تس...
3514    خامس اكبر اسباب الضعف الجنسي خامس احسن حلول تس...
3516    خامس اكبر اسباب الضعف الجنسي خامس احسن حلول تس...
3517    خامس اكبر اسباب الضعف الجنسي خامس احسن حلول تس...
3518    خامس اكبر اسباب الضعف الجنسي خامس احسن حلول تس...
Name: cleaned_transcript, Length: 1311, dtype: object

In [ ]:
import stanza
import pandas as pd
from collections import Counter
import re

# Download Arabic model if not already installed
stanza.download('ar')

# Initialize Stanza pipeline for POS tagging and NER
nlp = stanza.Pipeline('ar', processors='tokenize,pos,ner')

In [26]:
import stanza
import pandas as pd
from collections import Counter
import re
# Initialize cache
transcript_cache = {}  # {video_id: keywords_entities_list}

def process_transcript(video_id, transcript):
    """Process transcript with proper None handling"""
    if video_id in transcript_cache:
        return transcript_cache[video_id]
    
    try:
        doc = nlp(transcript)
        
        # Extract keywords (handle None lemmas)
        keywords = []
        for sentence in doc.sentences:
            for word in sentence.words:
                if word.upos in ['NOUN', 'ADJ']:
                    lemma = word.lemma if word.lemma is not None else word.text
                    keywords.append(lemma.lower())
        
        # Get top keywords (now with safe processing)
        top_keywords = [word for word, _ in Counter(keywords).most_common(50)]
        
        # Extract entities (with duplicate removal)
        entities = list({
            (ent.text.lower() if ent.text else "", ent.type)
            for ent in doc.ents
            if ent.type in ['PERSON', 'LOC', 'ORG'] and ent.text
        })
        
        # Combine and deduplicate
        combined = [e[0] for e in entities if e[0]] + top_keywords
        seen = set()
        keywords_entities = [x for x in combined if x and not (x in seen or seen.add(x))]
        
        transcript_cache[video_id] = keywords_entities
        return keywords_entities
    
    except Exception as e:
        print(f"Error processing video {video_id}: {str(e)}")
        return []

# Process data
video_data = df.drop_duplicates('video_id')[['video_id', 'cleaned_transcript']]
for _, row in video_data.iterrows():
    process_transcript(row['video_id'], row['cleaned_transcript'])

# Add to dataframe
df['keywords_entities'] = df['video_id'].map(transcript_cache)

In [30]:
df['comment'] = df['comment'].apply(clean_arabic_text)

In [33]:
df['comment_length'] = df['comment'].str.replace(r'\s', '', regex=True).str.len()


In [50]:
initial_count = len(df)
df = df[df['comment_length'] >= 4]  # Keep only comments with ≥4 letters
removed_count = initial_count - len(df)